# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d705f46048-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Sherlock Holmes explorative search") 


Consider the following exploratory scenario:


> Investigate the canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes) and which are the films and tv series related to the canon. We are also interested in the Emmy Awards and hiw many awards each film won. 


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q35610` | Arthur Conan Doyle        | node |



Also consider

```
wd:Q35610 ?p ?obj .
```

is the BGP to retrieve all **properties of Arthur Conan Doyle**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

2. Identify the BGP for film

3. Identify the BGP for television series

4. Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

5. Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

6. Identify the BGP for Primetime Emmy Award

7. Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

8. For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

## Task 1
Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

In [9]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4928'), ('name', 'Ricorso author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5714'), ('name', 'Tor.com author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P5470'), ('name', 'Scottish Poetry Library poet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http:/

In [16]:
# query example
queryString = """
SELECT DISTINCT ?p ?name ?obj ?value
WHERE {
   # bind something
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?value.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('name', 'father'), ('obj', 'http://www.wikidata.org/entity/Q342969'), ('value', 'Charles Altamont Doyle')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion'), ('obj', 'http://www.wikidata.org/entity/Q1069127'), ('value', 'Lapsed Catholic')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group'), ('obj', 'http://www.wikidata.org/entity/Q170826'), ('value', 'Irish people')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q208505'), ('value', 'crime novel')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q16082057'), ('value', "The New Student's Reference Work")]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q20664817'), ('value', 'detective novel')]
[('p', 'http://www.wikida

In [5]:
# query example
queryString = """
SELECT DISTINCT ?obj ?value
WHERE {
   # bind something
   wd:Q35610 wdt:P1455 ?obj .
   # get the label
   ?obj sc:name ?value.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4798323'), ('value', 'Arthur Conan Doyle bibliography')]
1


In [10]:
# query example
queryString = """
SELECT DISTINCT ?p ?name ?obj ?value
WHERE {
   # bind something
   wd:Q4798323 ?p ?obj .
   # get the label
   ?p sc:name ?name .
   ?obj sc:name ?value .
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1631107'), ('value', 'bibliography')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author'), ('obj', 'http://www.wikidata.org/entity/Q35610'), ('value', 'Arthur Conan Doyle')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q35610'), ('value', 'Arthur Conan Doyle')]
3


In [12]:
# query example
queryString = """
SELECT ?s ?value
WHERE {
   # bind something
   ?s wdt:P50 wd:Q35610 .
   # get the label
   ?s sc:name ?value.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17630018'), ('value', 'Your Share of Fear')]
[('s', 'http://www.wikidata.org/entity/Q22090538'), ('value', 'The Crime of the Congo')]
[('s', 'http://www.wikidata.org/entity/Q2316684'), ('value', 'canon of Sherlock Holmes')]
[('s', 'http://www.wikidata.org/entity/Q107410031'), ('value', 'The Adventure of the Dying Detective')]
[('s', 'http://www.wikidata.org/entity/Q466166'), ('value', 'The Adventure of the Dying Detective')]
[('s', 'http://www.wikidata.org/entity/Q733582'), ('value', 'His Last Bow')]
[('s', 'http://www.wikidata.org/entity/Q1192517'), ('value', 'The Adventure of the Greek Interpreter')]
[('s', 'http://www.wikidata.org/entity/Q3041819'), ('value', 'The Adventure of the Red Circle')]
[('s', 'http://www.wikidata.org/entity/Q45192'), ('value', 'The Hound of the Baskervilles')]
[('s', 'http://www.wikidata.org/entity/Q18546595'), ('value', 'Tales of Terror and Mystery')]
[('s', 'http://www.wikidata.org/entity/Q18546597'), ('valu

In [15]:
# query example
queryString = """
SELECT ?obj ?value
WHERE {
   # bind something
   wd:Q35610 wdt:P101 ?obj .
   # get the label
   ?obj sc:name ?value.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q208505'), ('value', 'crime novel')]
1


Final query for this task

In [14]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2316684 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('name', 'characters')]
[('p', 'http://www.wikidata.org/prop/direct/P6839'), ('name', 'TV Tropes identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P8895'), ('name', 'All the Tropes identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
11


## Task 2
Identify the BGP for film

In [31]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name ?obj ?value
WHERE {
   # bind something
   wd:Q2316684 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?value
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'), ('obj', 'http://www.wikidata.org/entity/Q1764019'), ('value', 'Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q733582'), ('value', 'His Last Bow')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q45192'), ('value', 'The Hound of the Baskervilles')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q277759'), ('value', 'book series')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q860831'), ('value', 'The Memoirs of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1631107'), ('value', 'bibliography')]
[('p', 'http://www.wikidata.or

In [28]:
queryString = """
SELECT DISTINCT ?s ?value ?obj ?name
WHERE {
   # bind something
   ?s wdt:P136 ?obj .
   # get the label
   ?s sc:name ?value .
   ?obj sc:name ?name .
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8204826'), ('value', 'Harpist of Keros'), ('obj', 'http://www.wikidata.org/entity/Q2586345'), ('name', 'Cycladic art')]
[('s', 'http://www.wikidata.org/entity/Q8195511'), ('value', 'Ayinde Barrister'), ('obj', 'http://www.wikidata.org/entity/Q3509353'), ('name', 'fuji music')]
[('s', 'http://www.wikidata.org/entity/Q15730544'), ('value', 'Crying for No Reason'), ('obj', 'http://www.wikidata.org/entity/Q1987980'), ('name', 'breakstep')]
[('s', 'http://www.wikidata.org/entity/Q15732538'), ('value', 'Chapliniana'), ('obj', 'http://www.wikidata.org/entity/Q4484381'), ('name', 'ballet film')]
[('s', 'http://www.wikidata.org/entity/Q15731615'), ('value', 'Scribblenauts Unmasked: A DC Comics Adventure'), ('obj', 'http://www.wikidata.org/entity/Q5370763'), ('name', 'emergent gameplay')]
[('s', 'http://www.wikidata.org/entity/Q104891471'), ('value', 'Ajin'), ('obj', 'http://www.wikidata.org/entity/Q61942616'), ('name', 'supernatural anime and mang

In [23]:
# Q4484381: ballet fim is sublass of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q4484381 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name .
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


Final query for this task

In [24]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11424 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1368'), ('name', 'LNB ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p',

## Task 3
Identify the BGP for television series

In [25]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?value
WHERE {
   # bind something
   ?s wdt:P279 wd:Q11424 .
   # get the label
   ?s sc:name ?value .
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5145881'), ('value', 'collage film')]
[('s', 'http://www.wikidata.org/entity/Q63302132'), ('value', 'film scored by this composer')]
[('s', 'http://www.wikidata.org/entity/Q4484381'), ('value', 'ballet film')]
[('s', 'http://www.wikidata.org/entity/Q3072043'), ('value', 'dance film')]
[('s', 'http://www.wikidata.org/entity/Q430525'), ('value', 'concert film')]
[('s', 'http://www.wikidata.org/entity/Q89928742'), ('value', 'non-existent film')]
[('s', 'http://www.wikidata.org/entity/Q663106'), ('value', 'buddy film')]
[('s', 'http://www.wikidata.org/entity/Q73504838'), ('value', 'Teenage Mutant Ninja Turtles film')]
[('s', 'http://www.wikidata.org/entity/Q18011171'), ('value', 'unfinished or abandoned film project')]
[('s', 'http://www.wikidata.org/entity/Q2143665'), ('value', "children's film")]
[('s', 'http://www.wikidata.org/entity/Q1935609'), ('value', 'propaganda film')]
[('s', 'http://www.wikidata.org/entity/Q102706436'), ('value', 'm

In [35]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?s wdt:P136 ?obj .
   # get the label
   ?s sc:name ?value .
   ?obj sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2586345'), ('name', 'Cycladic art')]
[('obj', 'http://www.wikidata.org/entity/Q3509353'), ('name', 'fuji music')]
[('obj', 'http://www.wikidata.org/entity/Q1987980'), ('name', 'breakstep')]
[('obj', 'http://www.wikidata.org/entity/Q4484381'), ('name', 'ballet film')]
[('obj', 'http://www.wikidata.org/entity/Q5370763'), ('name', 'emergent gameplay')]
[('obj', 'http://www.wikidata.org/entity/Q61942616'), ('name', 'supernatural anime and manga')]
[('obj', 'http://www.wikidata.org/entity/Q4049849'), ('name', 'spacesynth')]
[('obj', 'http://www.wikidata.org/entity/Q105334717'), ('name', 'progressive soul')]
[('obj', 'http://www.wikidata.org/entity/Q1108032'), ('name', 'Mumblecore')]
[('obj', 'http://www.wikidata.org/entity/Q4851628'), ('name', 'ballet')]
[('obj', 'http://www.wikidata.org/entity/Q1071873'), ('name', 'Chicago blues')]
[('obj', 'http://www.wikidata.org/entity/Q341364'), ('name', 'acid house')]
[('obj', 'http://www.wikidata.org/

In [34]:
# Q1366112 drama television series
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1366112 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name .
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q15416'), ('name', 'television program')]
[('obj', 'http://www.wikidata.org/entity/Q21010853'), ('name', 'drama')]
[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
3


Final query for this task

In [36]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q5398426 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9374'), ('name', 'ODLIS ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3219'), ('name', 'Encyclopædia Universalis ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), (

## Task 4
Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

In [8]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q5398426 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1154217'), ('name', 'Ironfist Chinmi')]
[('s', 'http://www.wikidata.org/entity/Q691528'), ('name', 'Wallace and Gromit')]
[('s', 'http://www.wikidata.org/entity/Q8193809'), ('name', 'Shomuni')]
[('s', 'http://www.wikidata.org/entity/Q33323'), ('name', 'Everybody Hates Chris')]
[('s', 'http://www.wikidata.org/entity/Q500769'), ('name', 'Freaks and Geeks')]
[('s', 'http://www.wikidata.org/entity/Q8350955'), ('name', 'Cordel Encantado')]
[('s', 'http://www.wikidata.org/entity/Q1145054'), ('name', 'The NBC Mystery Movie')]
[('s', 'http://www.wikidata.org/entity/Q344081'), ('name', 'Recess')]
[('s', 'http://www.wikidata.org/entity/Q13647845'), ('name', 'Grijpstra & De Gier')]
[('s', 'http://www.wikidata.org/entity/Q708661'), ('name', 'Lindenstraße')]
[('s', 'http://www.wikidata.org/entity/Q15099450'), ('name', 'Prime Minister and I')]
[('s', 'http://www.wikidata.org/entity/Q15476155'), ('name', 'The Full Sun')]
[('s', 'http://www.wikidata.org/

In [10]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   {
       ?s wdt:P31 wd:Q5398426 ;
          sc:name ?name.
   }
   UNION 
   {
       ?s wdt:P31 wd:Q11424 ;
          sc:name ?name.
   }
   FILTER (REGEX(?name, ".*Sherlock Holmes.*"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3481944'), ('name', 'Sherlock Holmes')]
[('s', 'http://www.wikidata.org/entity/Q7495083'), ('name', 'Sherlock Holmes')]
[('s', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes')]
[('s', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes')]
[('s', 'http://www.wikidata.org/entity/Q2292435'), ('name', 'Murder Rooms: Mysteries of the Real Sherlock Holmes')]
[('s', 'http://www.wikidata.org/entity/Q70777100'), ('name', 'Sherlock Holmes in Russia')]
[('s', 'http://www.wikidata.org/entity/Q2570112'), ('name', 'Sherlock Holmes')]
[('s', 'http://www.wikidata.org/entity/Q3959407'), ('name', 'Sherlock Holmes and Doctor Watson')]
[('s', 'http://www.wikidata.org/entity/Q2278328'), ('name', 'Sherlock Holmes')]
[('s', 'http://www.wikidata.org/entity/Q1269700'), ('name', 'The Rivals of Sherlock Holmes')]
[('s', 'http://www.wikidata.org/entity/Q20876053'), ('name', 'A Canine Sherlock Holmes')]
[('s', 'http://www.wikid

In [33]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q3959407 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
[('p', 'http://www.wikidata.org/prop/direct/P2603'), ('name', 'Kinopoisk film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2638'), ('name', 'TV.com ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P272'), ('name', 'production company')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3138'), ('name', 'OFDb film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P344'), ('name', 'director of photography')]
[('p', 'http://www.wi

In [38]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q21007878 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P1970'), ('name', 'MovieMeter film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2529'), ('name', 'ČSFD film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2603'), ('name', 'Kinopoisk film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2755'), ('name', 'exploitation visa number')]
[('p', 'http://www.wikidata.org/prop/direct/P2758'), ('name', 'CNC film rating (France)')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3143'), ('name', 'elFilm film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3203'), ('name', 'Ciné-Ressources fil

In [8]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?name ?year
WHERE {
   {
       ?s wdt:P31 wd:Q5398426 ;
          sc:name ?name.
        OPTIONAL {?s wdt:P580 ?year.}
   }
   UNION 
   {
       ?s wdt:P31 wd:Q11424 ;
          sc:name ?name.
        OPTIONAL {?s wdt:P577 ?year.}
   }
   FILTER (REGEX(?name, ".*Sherlock Holmes.*"))
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3481944'), ('name', 'Sherlock Holmes'), ('year', '1965-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q7495083'), ('name', 'Sherlock Holmes'), ('year', '1951-10-20T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('year', '2013-11-18T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('year', '1984-04-24T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q2292435'), ('name', 'Murder Rooms: Mysteries of the Real Sherlock Holmes'), ('year', '2000-04-04T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q70777100'), ('name', 'Sherlock Holmes in Russia'), ('year', '2020-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q2570112'), ('name', 'Sherlock Holmes'), ('year', '1954-10-18T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q3959407'), ('name', 'Sherlock Holmes and Doctor Watson'), ('year', '1979-01-01T00:00:00Z')]
[('s', 'http://w

Final query for this task

In [39]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name ?year
WHERE {
   {
       ?s wdt:P31 wd:Q5398426 ;
          sc:name ?name.
        OPTIONAL {?s wdt:P580 ?year.}
   }
   UNION 
   {
       ?s wdt:P31 wd:Q11424 ;
          sc:name ?name.
        OPTIONAL {?s wdt:P577 ?year.}
   }
   FILTER (REGEX(?name, ".*Sherlock Holmes.*"))
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3481944'), ('name', 'Sherlock Holmes'), ('year', '1965-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q7495083'), ('name', 'Sherlock Holmes'), ('year', '1951-10-20T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('year', '2013-11-18T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('year', '1984-04-24T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q2292435'), ('name', 'Murder Rooms: Mysteries of the Real Sherlock Holmes'), ('year', '2000-04-04T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q70777100'), ('name', 'Sherlock Holmes in Russia'), ('year', '2020-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q2570112'), ('name', 'Sherlock Holmes'), ('year', '1954-10-18T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q3959407'), ('name', 'Sherlock Holmes and Doctor Watson'), ('year', '1979-01-01T00:00:00Z')]
[('s', 'http://w

## Task 5
Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

In [43]:
# write your queries
queryString = """
SELECT DISTINCT ?actor ?name (COUNT(?films) AS ?film)
WHERE {
   {
       ?films wdt:P31 wd:Q5398426 ;
           wdt:P161 ?actor ;
          sc:name ?title.
   }
   UNION 
   {
       ?films wdt:P31 wd:Q11424 ;
          sc:name ?title.
   }
   FILTER (REGEX(?title, ".*Sherlock Holmes.*"))
   ?actor sc:name ?name.
}
GROUP BY ?actor ?name 
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q2057850'), ('name', 'Patrick Newell'), ('film', '1')]
[('actor', 'http://www.wikidata.org/entity/Q309589'), ('name', 'Peter Cushing'), ('film', '1')]
[('actor', 'http://www.wikidata.org/entity/Q2834347'), ('name', 'Oleksiy Gorbunov'), ('film', '1')]
[('actor', 'http://www.wikidata.org/entity/Q1351583'), ('name', 'Eric Porter'), ('film', '1')]
[('actor', 'http://www.wikidata.org/entity/Q5145239'), ('name', 'Colin Jeavons'), ('film', '1')]
[('actor', 'http://www.wikidata.org/entity/Q500404'), ('name', 'Maksim Matveyev'), ('film', '1')]
[('actor', 'http://www.wikidata.org/entity/Q517562'), ('name', 'Nigel Stock'), ('film', '1')]
[('actor', 'http://www.wikidata.org/entity/Q3031517'), ('name', 'Rosalie Williams'), ('film', '1')]
[('actor', 'http://www.wikidata.org/entity/Q4272897'), ('name', 'Andrey Lyovin'), ('film', '1')]
[('actor', 'http://www.wikidata.org/entity/Q2375210'), ('name', 'Andrei Panin'), ('film', '1')]
[('actor', 'http://ww

In [59]:
# write your queries
queryString = """
SELECT DISTINCT ?actor ?name (COUNT(?films) AS ?film)
WHERE {
       {?films wdt:P31 wd:Q5398426.}
       UNION
       {?films wdt:P31 wd:Q11424}
       FILTER (REGEX(?title, ".*Sherlock Holmes.*"))
       ?films wdt:P161 ?actor ;
              sc:name ?title.
   
       ?actor sc:name ?name.
}
GROUP BY ?actor ?name 
ORDER BY DESC (COUNT(?films))
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q336865'), ('name', 'Basil Rathbone'), ('film', '6')]
[('actor', 'http://www.wikidata.org/entity/Q728050'), ('name', 'Nigel Bruce'), ('film', '6')]
[('actor', 'http://www.wikidata.org/entity/Q467968'), ('name', 'Mary Gordon'), ('film', '5')]
[('actor', 'http://www.wikidata.org/entity/Q1344901'), ('name', 'Holmes Herbert'), ('film', '4')]
[('actor', 'http://www.wikidata.org/entity/Q3099595'), ('name', 'Gavin Muir'), ('film', '4')]
[('actor', 'http://www.wikidata.org/entity/Q1369906'), ('name', 'Henry Daniell'), ('film', '3')]
[('actor', 'http://www.wikidata.org/entity/Q3023171'), ('name', 'Dennis Hoey'), ('film', '3')]
[('actor', 'http://www.wikidata.org/entity/Q1276956'), ('name', 'Harry Cording'), ('film', '3')]
[('actor', 'http://www.wikidata.org/entity/Q236167'), ('name', 'Kelly Reilly'), ('film', '2')]
[('actor', 'http://www.wikidata.org/entity/Q2057850'), ('name', 'Patrick Newell'), ('film', '2')]
10


In [49]:
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   # bind something
   ?film wdt:P161 wd:Q440926 .
   # get the label
   ?film sc:name ?name .
   FILTER (REGEX(?name, ".*Sherlock Holmes.*"))
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes')]
[('film', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows')]
2


Final query for this task

In [52]:
# write your final query
queryString = """
SELECT DISTINCT ?actor ?name (COUNT(?films) AS ?film)
WHERE {
       {?films wdt:P31 wd:Q5398426.}
       UNION
       {?films wdt:P31 wd:Q11424}
       FILTER (REGEX(?title, ".*Sherlock Holmes.*"))
       ?films wdt:P161 ?actor ;
              sc:name ?title.
   
       ?actor sc:name ?name.
}
GROUP BY ?actor ?name 
HAVING (COUNT(?films) > 1)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q236167'), ('name', 'Kelly Reilly'), ('film', '2')]
[('actor', 'http://www.wikidata.org/entity/Q2057850'), ('name', 'Patrick Newell'), ('film', '2')]
[('actor', 'http://www.wikidata.org/entity/Q440926'), ('name', 'Geraldine James'), ('film', '2')]
[('actor', 'http://www.wikidata.org/entity/Q517562'), ('name', 'Nigel Stock'), ('film', '2')]
[('actor', 'http://www.wikidata.org/entity/Q180338'), ('name', 'Christopher Lee'), ('film', '2')]
[('actor', 'http://www.wikidata.org/entity/Q1369906'), ('name', 'Henry Daniell'), ('film', '3')]
[('actor', 'http://www.wikidata.org/entity/Q190386'), ('name', 'Rachel McAdams'), ('film', '2')]
[('actor', 'http://www.wikidata.org/entity/Q336865'), ('name', 'Basil Rathbone'), ('film', '6')]
[('actor', 'http://www.wikidata.org/entity/Q3023171'), ('name', 'Dennis Hoey'), ('film', '3')]
[('actor', 'http://www.wikidata.org/entity/Q7377388'), ('name', 'Rudolph Anders'), ('film', '2')]
[('actor', 'http://www.wi

## Task 6
Identify the BGP for Primetime Emmy Award

In [63]:
# Q1465984 is subclass of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1465984 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q768855'), ('name', 'orders, decorations, and medals of the United Kingdom')]
1


In [70]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q768855 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 3
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q618779'), ('name', 'award')]
[('obj', 'http://www.wikidata.org/entity/Q207333'), ('name', 'honor')]
[('obj', 'http://www.wikidata.org/entity/Q193622'), ('name', 'order')]
3


In [73]:
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q618779 .
   # get the label
   ?s sc:name ?name.
   FILTER(REGEX(?name, ".*Emmy Award.*"))
}
LIMIT 3
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18149577'), ('name', "Daytime Emmy Award for Outstanding Pre-School Children's Animated Program")]
[('s', 'http://www.wikidata.org/entity/Q18153303'), ('name', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('s', 'http://www.wikidata.org/entity/Q3002405'), ('name', 'Creative Arts Emmy Award')]
3


In [74]:
#Creative Arts Emmy Award is instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q3002405 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 3
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q618779'), ('name', 'award')]
[('obj', 'http://www.wikidata.org/entity/Q123737'), ('name', 'Emmy Award')]
2


In [77]:
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q123737 .
   # get the label
   ?s sc:name ?name.
   FILTER(REGEX(?name, ".*Emmy Award.*"))
}
LIMIT 3
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3002405'), ('name', 'Creative Arts Emmy Award')]
[('s', 'http://www.wikidata.org/entity/Q11247971'), ('name', 'News & Documentary Emmy Award')]
[('s', 'http://www.wikidata.org/entity/Q1044427'), ('name', 'Primetime Emmy Award')]
3


Final query for this task

In [78]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1044427 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('name', 'official website')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/P1027'), ('name', 'conferred by')]
[('p', 'http://www.wikidata.org/prop/direct/P2517'), ('name', 'category for recipients of this award')]
11


## Task 7
Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

In [86]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q1044427 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18153303'), ('name', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('s', 'http://www.wikidata.org/entity/Q28445729'), ('name', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q989439'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q2604800'), ('name', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('s', 'http://www.wikidata.org/entity/Q1277961'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('s', 'http://www.wikidata.org/entity/Q3123491'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q20714679'), ('name', 'Prim

In [88]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P279 wd:Q1044427 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

In [89]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q1044427 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18153303'), ('name', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('s', 'http://www.wikidata.org/entity/Q28445729'), ('name', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q989439'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q2604800'), ('name', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('s', 'http://www.wikidata.org/entity/Q1277961'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('s', 'http://www.wikidata.org/entity/Q3123491'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q20714679'), ('name', 'Prim

## Task 8
For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

In [90]:
# write your queries
queryString = """
SELECT DISTINCT ?films ?title (COUNT(?award) AS ?award)
WHERE {
       {?films wdt:P31 wd:Q5398426.}
       UNION
       {?films wdt:P31 wd:Q11424}
       FILTER (REGEX(?title, ".*Sherlock Holmes.*"))
       ?films wdt:P166 ?award ;
              sc:name ?title.
       ?award wdt:P31 wd:Q1044427 .
              
   
}
GROUP BY ?films ?title
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [93]:
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   {
       ?film wdt:P31 wd:Q5398426 ;
          sc:name ?name.
   }
   UNION 
   {
       ?film wdt:P31 wd:Q11424 ;
          sc:name ?name.
   }
   FILTER (REGEX(?name, ".*Sherlock Holmes.*"))
   ?film wdt:P166 ?award ;
              sc:name ?title .
   ?award wdt:P31 wd:Q1044427 .
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [106]:
#Q1285970
queryString = """
SELECT DISTINCT ?p ?value ?obj ?name
WHERE {
   # bind something
   wd:Q1285970 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
   ?p sc:name ?value.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('value', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q223033'), ('name', 'Sean Hayes')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('value', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q508043'), ('name', 'Eric Stonestreet')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('value', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q555236'), ('name', 'Don Knotts')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('value', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q465442'), ('name', 'Robert Guillaume')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('value', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q4538'), ('name', 'Jonathan Winters')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('value', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q77566'), ('name', 'Werner Klemperer')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('value', 'winner'), ('o

In [23]:
queryString = """
SELECT DISTINCT ?film ?name ?award ?title
WHERE {
   {
       ?film wdt:P31 wd:Q5398426 .
   }
   UNION 
   {
       ?film wdt:P31 wd:Q11424 .
          
   }
   ?film sc:name ?name ;
         wdt:P166 ?award .
   ?award  wdt:P31 wd:Q1044427 ;
           sc:name ?title .
   
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q500769'), ('name', 'Freaks and Geeks'), ('award', 'http://www.wikidata.org/entity/Q7243495'), ('title', 'Primetime Emmy Award for Outstanding Casting for a Comedy Series')]
[('film', 'http://www.wikidata.org/entity/Q1145054'), ('name', 'The NBC Mystery Movie'), ('award', 'http://www.wikidata.org/entity/Q3123491'), ('title', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('film', 'http://www.wikidata.org/entity/Q1145054'), ('name', 'The NBC Mystery Movie'), ('award', 'http://www.wikidata.org/entity/Q20714679'), ('title', 'Primetime Emmy Award for Outstanding Limited Miniseries')]
[('film', 'http://www.wikidata.org/entity/Q1145054'), ('name', 'The NBC Mystery Movie'), ('award', 'http://www.wikidata.org/entity/Q989447'), ('title', 'Primetime Emmy Award for Outstanding Lead Actress in a Miniseries or a Movie')]
[('film', 'http://www.wikidata.org/entity/Q1145054'), ('name', 'The NBC Mystery Movie'), ('award', 'http://w

In [21]:
queryString = """
SELECT DISTINCT ?p ?name ?obj ?value
WHERE {
   # bind something
   wd:Q4523158 ?p ?obj .
   # get the label
   ?p sc:name ?name .
   ?obj sc:name ?value
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q663106'), ('value', 'buddy film')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on'), ('obj', 'http://www.wikidata.org/entity/Q2316684'), ('value', 'canon of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q4360503'), ('value', 'Igor Petrenko')]
[('p', 'http://www.wikidata.org/prop/direct/P449'), ('name', 'original broadcaster'), ('obj', 'http://www.wikidata.org/entity/Q211511'), ('value', 'Russia-1')]
[('p', 'http://www.wikidata.org/prop/direct/P915'), ('name', 'filming location'), ('obj', 'http://www.wikidata.org/entity/Q159'), ('value', 'Russia')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin'), ('obj', 'http://www.wikidata.org/entity/Q159'), ('value', 'Russia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 

In [24]:
queryString = """
SELECT DISTINCT ?film ?name ?award ?title
WHERE {
   {
       ?film wdt:P31 wd:Q5398426 .
   }
   UNION 
   {
       ?film wdt:P31 wd:Q11424 .
          
   }
   ?film sc:name ?name ;
         wdt:P144 wd:Q2316684 ;
         wdt:P166 ?award .
   ?award  wdt:P31 wd:Q1044427 ;
           sc:name ?title .
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q17986500'), ('title', 'Primetime Emmy Award for Outstanding Cinematography for a Miniseries or Movie')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q16254850'), ('title', 'Primetime Emmy Award for Outstanding Sound Editing for a Limited or Anthology Series, Movie or Special')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q945887'), ('title', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q3403230'), ('title', 'Primetime Emmy Award for Outstanding Writing for a Miniseries, Movie or a Dramatic Special')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'She

In [25]:
queryString = """
SELECT DISTINCT ?film ?name ?award ?title
WHERE {
   {
       ?film wdt:P31 wd:Q5398426 .
   }
   UNION 
   {
       ?film wdt:P31 wd:Q11424 .
          
   }
   ?film sc:name ?name ;
         ?relation wd:Q2316684 ;
         wdt:P166 ?award .
   ?award  wdt:P31 wd:Q1044427 ;
           sc:name ?title .
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q17986500'), ('title', 'Primetime Emmy Award for Outstanding Cinematography for a Miniseries or Movie')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q16254850'), ('title', 'Primetime Emmy Award for Outstanding Sound Editing for a Limited or Anthology Series, Movie or Special')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q945887'), ('title', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q3403230'), ('title', 'Primetime Emmy Award for Outstanding Writing for a Miniseries, Movie or a Dramatic Special')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'She

In [28]:
queryString = """
SELECT DISTINCT  ?name ?relation ?relationType ?award ?title
WHERE {
   {
       ?film wdt:P31 wd:Q5398426 .
   }
   UNION 
   {
       ?film wdt:P31 wd:Q11424 .
          
   }
   ?film sc:name ?name ;
         ?relation wd:Q2316684 ;
         wdt:P166 ?award .
   ?award  wdt:P31 wd:Q1044427 ;
           sc:name ?title .
   ?relation sc:name ?relationType .
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Sherlock'), ('relation', 'http://www.wikidata.org/prop/direct/P144'), ('relationType', 'based on'), ('award', 'http://www.wikidata.org/entity/Q17986500'), ('title', 'Primetime Emmy Award for Outstanding Cinematography for a Miniseries or Movie')]
[('name', 'Sherlock'), ('relation', 'http://www.wikidata.org/prop/direct/P144'), ('relationType', 'based on'), ('award', 'http://www.wikidata.org/entity/Q16254850'), ('title', 'Primetime Emmy Award for Outstanding Sound Editing for a Limited or Anthology Series, Movie or Special')]
[('name', 'Sherlock'), ('relation', 'http://www.wikidata.org/prop/direct/P144'), ('relationType', 'based on'), ('award', 'http://www.wikidata.org/entity/Q945887'), ('title', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('name', 'Sherlock'), ('relation', 'http://www.wikidata.org/prop/direct/P144'), ('relationType', 'based on'), ('award', 'http://www.wikidata.org/entity/Q3403230'), ('title', 'Primetime Emmy A

Final query for this task

In [29]:
# write your final query
queryString = """
SELECT DISTINCT ?film ?name ?award ?title
WHERE {
   {
       ?film wdt:P31 wd:Q5398426 .
   }
   UNION 
   {
       ?film wdt:P31 wd:Q11424 .
          
   }
   ?film sc:name ?name ;
         ?relation wd:Q2316684 ;
         wdt:P166 ?award .
   ?award  wdt:P31 wd:Q1044427 ;
           sc:name ?title .
   ?relation sc:name ?relationType .
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q17986500'), ('title', 'Primetime Emmy Award for Outstanding Cinematography for a Miniseries or Movie')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q16254850'), ('title', 'Primetime Emmy Award for Outstanding Sound Editing for a Limited or Anthology Series, Movie or Special')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q945887'), ('title', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', 'http://www.wikidata.org/entity/Q3403230'), ('title', 'Primetime Emmy Award for Outstanding Writing for a Miniseries, Movie or a Dramatic Special')]
[('film', 'http://www.wikidata.org/entity/Q192837'), ('name', 'She